<a href="https://colab.research.google.com/github/james130625/notebook/blob/main/4_%EC%A1%B0%EA%B1%B4%EC%97%90_%EB%A7%9E%EB%8A%94_%EC%83%81%EC%9C%84_10%EA%B0%9C_KOSDAQ_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 패키지 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm -rf ~/.cache/matplotlib

# FinanceDataReader 설치
!pip install finance-datareader

# matplotlib, pandas 임포트 및 주피터 출력 설정, 한글 폰트 지정
import matplotlib.pyplot as plt
import FinanceDataReader as fdr
import pandas as pd
import numpy as np
from datetime import datetime

plt.rcParams['font.family'] = 'NanumBarunGothic'  # 한글 출력을 위한 폰트 설정

# KOSDAQ 상장 목록 가져오기
kosdaq_list = fdr.StockListing('KOSDAQ')

# 필터링 조건
price_lower = 5000
price_upper = 10000

# 필터링된 데이터프레임 저장용 리스트
filtered_stocks_data = []

# 오늘 날짜 구하기
today = datetime.today().strftime('%Y-%m-%d')

# 각 종목의 데이터를 가져와서 조건에 맞는지 확인
for code in kosdaq_list['Code']:
    try:
        # 종목 데이터 불러오기 (2023-01-02부터 오늘까지)
        data = fdr.DataReader(code, '2023-01-02', today)

        # 종가가 5000-10000 사이에 있는지 확인
        if data['Close'].iloc[-1] < price_lower or data['Close'].iloc[-1] > price_upper:
            continue

        # 거래량 평균 계산
        avg_volume = data['Volume'].mean()

        # 변동폭 계산 (종가의 표준편차)
        volatility = data['Close'].pct_change().std() * np.sqrt(252)

        # 최근 종가와 이동평균선 계산
        recent_close = data['Close'].iloc[-1]
        short_ma = data['Close'].rolling(window=5).mean()
        long_ma = data['Close'].rolling(window=20).mean()

        # Eating phase 확인
        if short_ma.iloc[-1] > long_ma.iloc[-1] and short_ma.iloc[-2] <= long_ma.iloc[-2]:
            filtered_stocks_data.append({
                'Code': code,
                'Current Price': recent_close,
                'Avg Volume': avg_volume,
                'Volatility': volatility
            })
    except Exception as e:
        # 데이터 불러오기에 실패하면 무시
        print(f"Failed to process {code}: {e}")
        continue

# 필터링된 종목 데이터프레임 생성
filtered_stocks_df = pd.DataFrame(filtered_stocks_data)

# 거래량이 많고 변동성이 높은 순으로 정렬
filtered_stocks_df = filtered_stocks_df.sort_values(by=['Avg Volume', 'Volatility'], ascending=[False, False])

# 상위 10개 종목 선정
top_10_stocks_df = filtered_stocks_df.head(10)

# 결과 출력
print("조건에 맞는 상위 10개 종목 목록:")
print(top_10_stocks_df)

# 필터링된 종목 리스트 데이터프레임 출력 함수
def display_dataframe_to_user(name: str, dataframe: pd.DataFrame) -> None:
    from IPython.display import display, HTML
    display(HTML(f"<h2>{name}</h2>"))
    display(dataframe)

display_dataframe_to_user(name="조건에 맞는 상위 10개 KOSDAQ 종목 목록", dataframe=top_10_stocks_df)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (27.9 MB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/

,Code,Current Price,Avg Volume,Volatility
0,298830,6770,1.912986e+06,0.881411
1,200470,5940,1.462729e+06,0.714878
4,038870,5560,1.114004e+06,0.756328
5,317850,9120,5.625194e+05,0.536083
2,297090,8640,4.539023e+05,0.531260
3,041440,6790,4.516641e+05,0.350993
